In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['PORJ_LIB'] = '/home/jhemedinger/anaconda3/envs/goes_env/share/proj'

import pyart
from pyart.core import geographic_to_cartesian_aeqd, Grid
import cartopy.crs as ccrs
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import netCDF4
import itertools
import glob
from scipy import interpolate


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
def _nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def interp_lonlat(lon, lat, data, radar_lon, radar_lat, grid_x, grid_y):
    x, y = geographic_to_cartesian_aeqd(lon, lat, radar_lon, radar_lat)

    target_x, target_y = np.meshgrid(grid_x, grid_y)
    points = list(zip(x.flatten(), y.flatten()))
    values = data.flatten()
    interp_data = interpolate.griddata(points, values, (target_x, target_y))
    interp_data = ma.masked_where(np.isnan(interp_data), interp_data)
    interp_data = np.tile(interp_data, (2, 1, 1))
    interp_data = interp_data[np.newaxis, :, :, :]
    return interp_data

In [3]:
def get_grid(filename, grid_x, grid_y, grid_z, output_dir):
    print('Gridding... ' + filename)
    
    # Reading in the GOES netCDF files
    nc = netCDF4.Dataset(filename)
    
    
    sat_height = nc.variables['goes_imager_projection'].perspective_point_height
    _x = nc.variables['x'] * sat_height
    _y = nc.variables['y'] * sat_height
    _c = nc.variables['CMI_C13'][:] * -1
    
    data = nc.variables['CMI_C13']
    
    proj_var = nc.variables[data.grid_mapping]
    
    globe = ccrs.Globe(ellipse='sphere', semimajor_axis=proj_var.semi_major_axis,
                       semiminor_axis=proj_var.semi_minor_axis)
    
    proj = ccrs.Geostationary(central_longitude=-75,sweep_axis='x',
                              satellite_height=sat_height, globe = globe)
    
    trans = ccrs.PlateCarree(central_longitude=0)
    
    transform_xy = trans.transform_points(proj, _x, _y)
    
    lim = [_nearest(transform_xy[:,0],-103),_nearest(transform_xy[:,0],-92)
            ,_nearest(transform_xy[:,1],42),_nearest(transform_xy[:,1],30)]
    
    
    x = _x[lim[0]:lim[1]]
    y = _y[lim[2]:lim[3]]
    
    c = _c[lim[2]:lim[3],lim[0]:lim[1]]
        
    x_mesh, y_mesh = np.meshgrid(x, y)
    
    lonlat = trans.transform_points(proj, x_mesh, y_mesh)
    lons = lonlat[:, :, 0]
    lats = lonlat[:, :, 1]
    
    # Values are from the KVNX radar
    # KVNX data was used becuase original project this was used for 
    # was using that radar to compare GOES data to the radar data
    radar_lon = -98.12771606
    radar_lat = 36.74061584
    radar_alt = 383.0
    
    interp_c = interp_lonlat(lons, lats, c,
                             radar_lon, radar_lat, grid_x, grid_y) 
    
    # Creating the data dictonaries needed for the PyART grid function
    # radar_latitude, radar_longitude, and radar_altitude are not required for
    # gridding, but to work with TINT they are needed. 
    _time = {'calendar': 'gregorian','data': np.array([ 0.934]),
             'long_name': 'Time of grid', 'standard_name': 'time',
             'units': str('seconds since ' + nc.time_coverage_end)}
    
#    _fields = {'reflectivity': {'_FillValue': -9999.0, 'data': ma.masked_array(c, mask= False),
#                       'long_name': 'reflectivity',
#                       'standard_name': 'equivalent_reflectivity_factor',
#                       'units': 'dBZ', 'valid_max': c.max(), 'valid_min': c.min()}}
    
    _fields = {'c13': {'_FillValue': -9999.0,
                       'data': interp_c,
                       'long_name': 'channel 13 10.3 microns K',
                       'standard_name': 'c13',
                       'units': 'K', 'valid_max': c.max(),
                       'valid_min': c.min()}}
    
    _metadata = {'Conventions': '', 'comment': '',
                 'history': '', 'institution': '', 'instrument_name': '',
                 'original_container': 'NEXRAD Level II', 'references': '',
                 'source': '', 'title': '', 'vcp_pattern': '', 'version': ''}
    
    _origin_latitude = {'data': ma.array([radar_lat]),
                        'long_name': 'Latitude at grid origin',
                        'standard_name': 'latitude',
                        'units': 'degrees_north', 'valid_max': 90.0,
                        'valid_min': -90.0}
    
    _radar_latitude = {'data': ma.array([radar_lat]),
                       'long_name': 'Latitude of radar used to make the grid',
                       'units': 'degrees_north',
                       'standard_name': 'latitude'}
    
    _origin_longitude = {'data': ma.array([radar_lon]), 
                         'long_name': 'Longitude at grid origin', 
                         'standard_name': 'longitude', 'units': 'degrees_east', 
                         'valid_max': 180.0, 'valid_min': -180.0}
    
    _radar_longitude = {'data': ma.array([radar_lon]),
                        'standard_name': 'longitude',
                        'long_name': 'Longitude of radars used to make the grid',
                        'units': 'degrees_east'}
    
    _origin_altitude = {'data': ma.array([radar_alt]), 
                        'long_name': 'Altitude at grid origin', 
                        'standard_name': 'altitude', 'units': 'm'}
    
    _radar_altitude = {'data': ma.array([radar_alt]),
                       'long_name': 'Altitude of radars used to make the grid',
                       'units': 'm', 'standard_name': 'altitude'}
    
    _x = {'axis': 'X', 'data': grid_x, 
          'long_name': 'X distance on the projection plane from the origin', 
          'standard_name': 'projection_x_coordinate', 'units': 'm'}
    
    _y = {'axis': 'Y', 'data': grid_y, 
          'long_name': 'Y distance on the projection plane from the origin', 
          'standard_name': 'projection_x_coordinate', 'units': 'm'}
    
#     _z = {'axis': 'Z', 'data': grid_z,
#           'long_name': 'Z distance on the projection plane from the origin',
#           'positive': 'up', 'standard_name': 'projection_z_coordinate',
#           'units': 'm'}
    
    _z = {'axis': 'Z', 'data': ma.array([0, grid_z]),
          'long_name': 'Z distance on the projection plane from the origin',
          'positive': 'up', 'standard_name': 'projection_z_coordinate',
          'units': 'm'}
    
    # Creating and saving GOES grid using PyARTs gridding function
    # Grid documentation can be found here:
    # https://github.com/ARM-DOE/pyart/blob/master/pyart/core/grid.py
    grid = Grid(time=_time, fields=_fields, metadata=_metadata,
                origin_latitude=_origin_latitude, origin_longitude=_origin_longitude,
                origin_altitude=_origin_altitude, x=_x, y=_y, z=_z,
                radar_longitude=_radar_longitude, radar_latitude=_radar_latitude,
                radar_altitude=_radar_altitude)
    
    grid_name = os.path.basename(filename[:-3] + '_grid.nc')
    full_name = os.path.join(output_dir, grid_name)
    pyart.io.write_grid(full_name, grid)

In [4]:
# Setting the variables needed for get_grid and then creating grids
# grid_x, grid_y, grid_z can be changed to any values those are just what we used
grid_x = np.linspace(-200000,200000,500)
grid_y = np.linspace(-200000,200000,500)
# grid_z = np.linspace(0,15000,31)
grid_z = grid_x[1] - grid_x[0]
output_dir = '/home/jhemedinger/suli_projects/precipitation-onset/grids/sat_grids'

# Pulling the GOES data that is going to be gridded using glob
filename = glob.glob('/home/jhemedinger/suli_projects/precipitation-onset/data/data/*')
filename.sort()

# For loop for gridding the GOES data
for file in filename:
    get_grid(filename=file, grid_x=grid_x, 
             grid_y=grid_y, grid_z=grid_z, 
             output_dir=output_dir)
print('Gridding Complete')

Gridding... /home/jhemedinger/suli_projects/precipitation-onset/data/data/OR_ABI-L2-MCMIPM1-M3_G16_s20181741700341_e20181741700399_c20181741700473.nc
Gridding... /home/jhemedinger/suli_projects/precipitation-onset/data/data/OR_ABI-L2-MCMIPM1-M3_G16_s20181741701341_e20181741701399_c20181741701470.nc
Gridding... /home/jhemedinger/suli_projects/precipitation-onset/data/data/OR_ABI-L2-MCMIPM1-M3_G16_s20181741702341_e20181741702399_c20181741702471.nc
Gridding... /home/jhemedinger/suli_projects/precipitation-onset/data/data/OR_ABI-L2-MCMIPM1-M3_G16_s20181741703341_e20181741703399_c20181741703469.nc
Gridding... /home/jhemedinger/suli_projects/precipitation-onset/data/data/OR_ABI-L2-MCMIPM1-M3_G16_s20181741704341_e20181741704399_c20181741704469.nc
Gridding... /home/jhemedinger/suli_projects/precipitation-onset/data/data/OR_ABI-L2-MCMIPM1-M3_G16_s20181741705341_e20181741705399_c20181741705473.nc
Gridding... /home/jhemedinger/suli_projects/precipitation-onset/data/data/OR_ABI-L2-MCMIPM1-M3_G16_s

Gridding... /home/jhemedinger/suli_projects/precipitation-onset/data/data/OR_ABI-L2-MCMIPM1-M3_G16_s20181741755341_e20181741755399_c20181741755468.nc
Gridding... /home/jhemedinger/suli_projects/precipitation-onset/data/data/OR_ABI-L2-MCMIPM1-M3_G16_s20181741756341_e20181741756399_c20181741756472.nc
Gridding... /home/jhemedinger/suli_projects/precipitation-onset/data/data/OR_ABI-L2-MCMIPM1-M3_G16_s20181741757341_e20181741757399_c20181741757473.nc
Gridding... /home/jhemedinger/suli_projects/precipitation-onset/data/data/OR_ABI-L2-MCMIPM1-M3_G16_s20181741758341_e20181741758399_c20181741758470.nc
Gridding... /home/jhemedinger/suli_projects/precipitation-onset/data/data/OR_ABI-L2-MCMIPM1-M3_G16_s20181741759341_e20181741759399_c20181741759471.nc
Gridding Complete
